In [2]:
from typing import Set


CNF = Set[Set[int]]
LIT = Set[int]

In [25]:
def sgn(x):
    return 1 if x >= 0 else 0


def remove_lits(S: CNF, lits: Set[int]):
    for C in S:
        if C & lits:
            S.remove(C)
            S.add(C - lits)

def get_single(C: Set[int]):
    return next(iter(C))

def unit_clause(S: CNF, num_lit: int):
    #   if there is some clause consisting of a single literal l (a
    #  unit clause), remove instances of −l from other clauses and then remove any
    #  clauses containing l (including the unit clause itself);
    singles = {get_single(C) for C in S if len(C) == 1}
    remove_lits(S, {-l for l in singles})
    return {C for C in S if not (C - singles)}


def pure_literal(S: CNF, num_lit: int):
    #  if there is a literal appearing in the formula only positively
    #  or only negatively, remove all clauses containing it
    counts = [[0, 0] for _ in range(num_lit)]
    for C in S:
        for l in C:
            counts[sign(l)] += 1

    lits_to_remove = {e for e, c in enumerate(counts) if c[0] == 0 or c[1] == 0}

    remove_lits(S, lits_to_remove)
    return S


def tautology(S: CNF, num_lit: int):
    #   if some clause contains complementary literals, l and −l,
    #  remove this clause;
    return {C for C in S if not (l in C and -l in C)}


def resolution(S: CNF, num_lit: int):
    #  choose a literal l and split the set of clauses S into a
    #  set S1 of clauses containing l only positively, a set S2 of clauses containing l only
    #  negatively, and the rest S3, and a new set of clauses is {c1 sum c2 \ {l, -l} for c1, c2 in s1, s2} sum s3
    S1 = {C for C in S if l in C and -l not in C}
    S2 = {C for C in S if -l in C and l not in C}
    S3 = S - S1 - S2

    return {(c1 | c2) - {l, -l} for c1, c2 in zip(S1, S2)} | S3


def check_for_empty(S: CNF):
    return any(not C for C in S)


def DF(S: CNF, num_lit: int):
    rules = [unit_clause, pure_literal, tautology, resolution]

    idx = 0
    while True:
        if not S:
            return "SAT"

        if check_for_empty(S):
            return "UNSAT"

        S = rules[idx](S, num_lit)
        print("Rule ", rules[idx])
        idx = (idx + 1) % len(rules)

In [26]:
fname = "./ex.cnf"
num_lit, _, data = input_data(fname)

DF(data, num_lit)

Rule  <function unit_clause at 0x7fda60d6e598>


'SAT'

In [27]:
data

{frozenset({-1, 2, 3}), frozenset({-3, 1})}